# Restore ROI data to OMERO from Database files

## Laura Cooper, 15/06/2021

Read text files generated from database and import recoved ROIs to the associated images. Use for the case where images/ROIs have been accidentally deleted.

### Import packages

In [ ]:
import pandas as pd
import omero.clients
import getpass
from omero.gateway import BlitzGateway

### Read database files

In [ ]:
def read_dbfile(file):
    '''
    Read and format database file
    Input: file, string
    Output: df, dataframe
    '''
    df =  pd.read_csv(file, sep='|'  , engine='python')
    df=df.drop(axis=0,index=0)
    df=df.reset_index(drop=True)
    df=df.rename(columns=lambda x: x.strip())
    return df

In [ ]:
id_filename=read_dbfile('id_filename.txt')
id_filename['id']=id_filename['id'].astype(int)
roiID_perimage=read_dbfile('roiID_perimage.txt')
roiID_perimage['id']=roiID_perimage['id'].astype(int)
roiID_perimage['image']=roiID_perimage['image'].astype(int)
shapesPerROI=read_dbfile('shapesPerROI.txt')
#shapesPerROI['thet']=shapesPerROI['thet'].astype(str).astype(int)
shapesPerROI['roi']=shapesPerROI['roi'].astype(int)
shapesPerROI['discriminator']=shapesPerROI['discriminator'].str.strip()
shapesPerROI['thet']=shapesPerROI['thet'].str.strip()

### Connect to OMERO and import required tools

In [ ]:
USERNAME=input('username: ')
PASSWORD=getpass.getpass('password: ')
SERVER=input('server: ')
conn = BlitzGateway(USERNAME, PASSWORD, host=SERVER, port=4064, secure=True)
conn.connect()

In [ ]:
updateService = conn.getUpdateService()
from omero.rtypes import rdouble, rint, rstring

In [ ]:
# We have a helper function for creating an ROI and linking it to new shapes
def create_roi(img, shapes):
    '''
    Add ROI to an image in OMERO
    Input: img, OMERO image object
           shapes, OMERO shape object(s)
    '''
    # create an ROI, link it to Image
    roi = omero.model.RoiI()
    # use the omero.model.ImageI that underlies the 'image' wrapper
    roi.setImage(img._obj)
    for shape in [shapes]:
        roi.addShape(shape)
    # Save the ROI (saves any linked shapes too)
    return updateService.saveAndReturnObject(roi)

### Get list of images to restore ROIs to

In [ ]:
dataset_id=1000
images = conn.getObjects("Image", opts={"dataset": dataset_id})
#for image in images:
    #print(image.getName())
    #print(image.getId())

### Add the ROIs to the images
This loop uses the image name to find the equivalent image ID from the database files and then finds all the ROIs associated to this image. The ROI parameters are read from another database file and are used to create a new shape. This shape is then imported into OMERO to the correct image.

In [ ]:
for image in images:
    image_name=image.getName()
    print(image_name)
    New_ID=image.getId()
    Old_ID=id_filename[id_filename['name'].str.contains(image_name)]['id'].values[0]
    rois=roiID_perimage[roiID_perimage['image'].isin([Old_ID])]['id']
    for roi in rois:
        shape=shapesPerROI[shapesPerROI['roi'].isin([roi])]
        if shape['discriminator'].values=='ellipse':
            ellipse = omero.model.EllipseI()
            ellipse.x = rdouble(shape['x'].values)
            ellipse.y = rdouble(shape['y'].values)
            ellipse.radiusX = rdouble(shape['radiusx'].values)
            ellipse.radiusY = rdouble(shape['radiusy'].values)
            #ellipse.theZ = rint(z)
            ellipse.theT = rint(shape['thet'].values)
            ellipse.textValue = rstring(shape['textvalue'].values)
            img = conn.getObject("Image", New_ID)
            create_roi(img, ellipse)
            del(ellipse)

In [ ]:
conn.close()